# Required Libraries

In [1]:
# tensorflow==2.3.1
# nltk==3.5
# colorama==0.4.3
# numpy==1.18.5
# scikit_learn==0.23.2
# Flask==1.1.2

# Import Library

In [2]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# Uplaod data 

In [3]:
# Loading data from 'intents.json'
with open('intents.json') as file:
    intents = json.load(file)

# Loading data from 'Intent.json' with 'utf-8' encoding
with open('Intent.json', encoding='utf-8') as file:
    Intent = json.load(file)

# Loading data from 'intents_arabic.json' with 'utf-8' encoding
with open('intents_arabic.json', encoding='utf-8') as file:
    intents_arabic = json.load(file)

# Loading data from 'country.json'
with open("country.json") as file:
    country = json.load(file)

# Printing loaded data from each file
print(intents)
print(Intent)
print(intents_arabic)
print(country)


{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hi :)', 'Hey :-)', "I'm fine, what can I do for you?", 'Hi there, how can I help?', 'Hello', 'Thanks for visiting']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'See you soon', 'See you later'], 'responses': ['Have a nice day', 'See you later, thanks for visiting', 'Goodbye!', 'Bye, see you soon!', 'Bye! Come back again soon.', 'Take care!', 'Have a great day!']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!", 'Thanks for the help'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure', "You're most welcome!"]}, {'tag': 'about', 'patterns': ['Who are you?', 'What are you?', 'Who you are?', 'Who you doing?', 'What are you doing?'], 'responses': ["I'm H, an Artificial Intelligent bot", "I'm Hesho, your bot assistant", "I'm H, your friend!", 'I do what you need in app?']}, {'tag': 'name',

In [4]:
# Iterating through each 'intent' in the 'intents' list inside the 'country' dictionary
for intent in country['intents']:
    # Iterating through the range of indices of 'patterns' in each 'intent'
    for pattern_index in range(len(intent['patterns'])):
        # Accessing each 'pattern' in the 'patterns' list of the current 'intent'
        pattern = intent['patterns'][pattern_index]
        # Removing specific characters ('{', '!', '%', '?') and converting to lowercase
        cleaned_pattern = pattern.replace('{', '').replace('!', '').replace('%', '').replace('?', '').lower()
        # Replacing the original pattern with the cleaned pattern in the 'patterns' list
        intent['patterns'][pattern_index] = cleaned_pattern

# Displaying the modified 'country' data in JSON format with indentation for readability
print(json.dumps(country, indent=4))


{
    "intents": [
        {
            "tag": "capital_query",
            "patterns": [
                "what is the capital of albania",
                "what is the capital of algeria",
                "what is the capital of andorra",
                "what is the capital of angola",
                "what is the capital of antigua and barbuda",
                "what is the capital of argentina",
                "what is the capital of armenia",
                "what is the capital of australia",
                "what is the capital of austria",
                "what is the capital of azerbaijan",
                "what is the capital of bahamas",
                "what is the capital of bahrain",
                "what is the capital of bangladesh",
                "what is the capital of barbados",
                "what is the capital of belarus",
                "what is the capital of belgium",
                "what is the capital of belize",
                "what is the capital o

# Consolidating_Data

In [5]:
training_sentences = []
training_labels = []
labels = []
responses = []

# Data 1 Processing (intents)
for intent in intents['intents']:
    # Collecting patterns for sentences
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    # Collecting responses
    responses.append(intent['responses'])

    # Adding unique tags to the labels list
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

# Data 2 Processing (intents)
for intent in intents['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

# Data 3 Processing (country)
for intent in country['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

# Data 4 Processing (intents_arabic)
for intent in intents_arabic['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    # Appending only if the tag is not already in the labels list
    if intent['tag'] in labels:
        labels.append(intent['tag'])

# Number of unique classes (intents or labels)
num_classes = len(labels)
num_classes


27

 1. Initialization: Initializes empty lists: training_sentences, training_labels, labels, and responses.

2. Data Processing - Intents 1: Processes data related to intents from the intents source:
   Iterates through each intent in the 'intents' data.
   Collects patterns (sentences) and corresponding tags (labels) from each intent.
   Gathers responses associated with each intent.
   Adds unique tags to the labels list if they're not already present.
3. Data Processing - Intents 2: Processes another set of data related to intents from the same intents source:

    Similar to the first intent processing, it appends patterns, labels, and responses to their respective lists.
    Checks and adds unique tags to the labels list.
4. Data Processing - Country: Processes data related to 'country' intents:

    Gathers patterns, labels, and responses from the 'country' intents.
    Adds unique tags to the labels list if they're not already present.
5. Data Processing - Intents Arabic: Processes data related to intents in Arabic:

    Gathers patterns, labels, and responses from the 'intents_arabic' source.
    Appends tags to the labels list if they are already present.
6. Counting Unique Classes: Determines the number of unique classes (intents or labels) by calculating the length of the labels list, which represents the unique tags gathered from the processed data sources.

# Label_Encoder 

In [6]:
# Initialize a LabelEncoder object
le = LabelEncoder()

# Fit the LabelEncoder to the training labels to transform them into numerical labels
le.fit(training_labels)

# Transform the original training labels into numerical labels
training_labels = le.transform(training_labels)

# Display the transformed training labels
training_labels


array([10, 10, 10, 10, 10, 10,  9,  9,  9,  9,  9, 17, 17, 17, 17, 17,  0,
        0,  0,  0,  0, 13, 13, 13, 11, 11, 11, 11, 11, 11, 11,  6,  6,  6,
        6, 15, 15, 15, 15,  3,  3,  3,  3,  7,  7,  7, 16, 16, 16, 16,  4,
        4,  4,  4, 12, 12, 12, 14, 14, 14,  1,  1,  1,  8,  8,  8,  5,  5,
        5, 10, 10, 10, 10, 10, 10,  9,  9,  9,  9,  9, 17, 17, 17, 17, 17,
        0,  0,  0,  0,  0, 13, 13, 13, 11, 11, 11, 11, 11, 11, 11,  6,  6,
        6,  6, 15, 15, 15, 15,  3,  3,  3,  3,  7,  7,  7, 16, 16, 16, 16,
        4,  4,  4,  4, 12, 12, 12, 14, 14, 14,  1,  1,  1,  8,  8,  8,  5,
        5,  5,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2

1. LabelEncoder Initialization: Creates a LabelEncoder object named le.

2. Fitting the LabelEncoder: The fit() method of the LabelEncoder is used to fit the encoder to the training_labels. This step basically identifies unique labels in training_labels and assigns a unique numerical value to each unique label.

3. Transforming Training Labels: The transform() method of the LabelEncoder is applied to training_labels. It replaces each label in training_labels with its corresponding encoded numerical value obtained from the fitting step.

3. Display Transformed Labels: The training_labels variable now contains the numerical representations of the original categorical labels after transformation using the LabelEncoder. The code snippet concludes by displaying the transformed labels in the output.

# Text Tokenization and Sequence Padding

In [7]:
# Setting parameters
vocab_size = 1000  # Maximum number of words to keep in the vocabulary
embedding_dim = 16  # Dimension of the word embedding
max_len = 20  # Maximum length of sequences
oov_token = "<OOV>"  # Out-of-vocabulary token

# Initializing a Tokenizer object with specified parameters
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)

# Creating a word index based on the training sentences
tokenizer.fit_on_texts(training_sentences)

# Retrieving the word index generated by the Tokenizer
word_index = tokenizer.word_index

# Converting text sentences to sequences of integers using the word index
sequences = tokenizer.texts_to_sequences(training_sentences)

# Padding sequences to ensure uniform length (truncating longer sequences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)


1. Setting Parameters: Defines parameters such as vocabulary size (vocab_size), embedding dimension (embedding_dim), maximum sequence length (max_len), and an out-of-vocabulary token (oov_token).

2. Tokenizer Initialization: Creates a Tokenizer object with the specified parameters (limited vocabulary size and out-of-vocabulary token).

3. Fitting Tokenizer: fit_on_texts() method tokenizes and builds the word index based on the training sentences provided (training_sentences).

4. Word Index Retrieval: Retrieves the word index generated by the Tokenizer, containing the mapping of words to numerical indices.

5. Text to Sequences: Converts text sentences (training_sentences) to sequences of integers using the word index obtained from the Tokenizer.

6. Sequence Padding: Pads sequences to ensure uniform length (max_len). Longer sequences are truncated from the end (truncating='post') to match the specified maximum length.

# Model

In [8]:
# Create a Sequential model
model = Sequential()

# Add an Embedding layer with specified parameters to the model
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))

# Add GlobalAveragePooling1D layer to the model
model.add(GlobalAveragePooling1D())

# Add a Dense layer with 16 units and ReLU activation function to the model
model.add(Dense(16, activation='relu'))

# Add another Dense layer with 16 units and ReLU activation function to the model
model.add(Dense(16, activation='relu'))

# Add a Dense output layer with 'num_classes' units and softmax activation function for classification
model.add(Dense(num_classes, activation='softmax'))

# Compile the model using sparse categorical crossentropy loss, Adam optimizer, and accuracy metric
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Display the summary of the model architecture
model.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 27)                459       
                                                                 
Total params: 17003 (66.42 KB)
Trainable params: 17003 (66.42 KB)
Non-trainable params: 0 (0.00 Byte)
__________________

1. Sequential Model Creation: Initializes a Sequential model.

2. Embedding Layer: Adds an Embedding layer to the model. This layer converts words into fixed-size dense vectors (word embeddings). Parameters like vocab_size (size of the vocabulary), embedding_dim (dimension of the embeddings), and input_length (length of input sequences) are specified.

3. GlobalAveragePooling1D Layer: Adds a Global Average Pooling 1D layer to the model. This layer reduces the dimensionality of the output from the Embedding layer by taking the average of all the embeddings. It creates a fixed-length output regardless of the input sequence length.

4. Dense Layers: Adds two Dense layers with 16 units each and ReLU activation functions. These dense layers help in learning complex patterns in the data.

5. utput Layer: Adds a Dense output layer with num_classes units (representing the number of unique classes in the classification task) and uses a softmax activation function. Softmax ensures that the output values represent probabilities for each class.

6. Model Compilation: Compiles the model with settings for loss, optimizer, and metrics.

   Loss function: Sparse Categorical Crossentropy - suitable for multi-class classification tasks with integer labels.
   Optimizer: Adam - an efficient and widely used optimization algorithm.
   Metrics: Accuracy - to measure the model's performance during training.
7. Model Summary: Displays a summary of the model architecture, including the layers, their types, output shapes, and trainable parameters.

# Training_Model

In [9]:
epochs = 500
history = model.fit(
    padded_sequences,
    np.array(training_labels),  
    epochs=epochs
)

Epoch 1/500


12/12 [==============================] - 4s 7ms/step - loss: 3.2794 - accuracy: 0.4262
Epoch 2/500
12/12 [==============================] - 0s 6ms/step - loss: 3.2396 - accuracy: 0.5404
Epoch 3/500
12/12 [==============================] - 0s 5ms/step - loss: 3.1867 - accuracy: 0.5404
Epoch 4/500
12/12 [==============================] - 0s 7ms/step - loss: 3.1107 - accuracy: 0.5404
Epoch 5/500
12/12 [==============================] - 0s 6ms/step - loss: 2.9994 - accuracy: 0.5404
Epoch 6/500
12/12 [==============================] - 0s 6ms/step - loss: 2.8423 - accuracy: 0.5404
Epoch 7/500
12/12 [==============================] - 0s 7ms/step - loss: 2.6348 - accuracy: 0.5404
Epoch 8/500
12/12 [==============================] - 0s 7ms/step - loss: 2.3697 - accuracy: 0.5404
Epoch 9/500
12/12 [==============================] - 0s 9ms/step - loss: 2.1090 - accuracy: 0.5404
Epoch 10/500
12/12 [==============================] - 0s 6ms/step - loss: 1.9435 - accuracy: 0.5404
Epoch 1

12/12 [==============================] - 0s 7ms/step - loss: 0.1412 - accuracy: 0.9833
Epoch 160/500
12/12 [==============================] - 0s 9ms/step - loss: 0.1380 - accuracy: 0.9833
Epoch 161/500
12/12 [==============================] - 0s 6ms/step - loss: 0.1352 - accuracy: 0.9833
Epoch 162/500
12/12 [==============================] - 0s 7ms/step - loss: 0.1307 - accuracy: 0.9833
Epoch 163/500
12/12 [==============================] - 0s 8ms/step - loss: 0.1293 - accuracy: 0.9861
Epoch 164/500
12/12 [==============================] - 0s 7ms/step - loss: 0.1253 - accuracy: 0.9889
Epoch 165/500
12/12 [==============================] - 0s 7ms/step - loss: 0.1210 - accuracy: 0.9833
Epoch 166/500
12/12 [==============================] - 0s 7ms/step - loss: 0.1186 - accuracy: 0.9889
Epoch 167/500
12/12 [==============================] - 0s 7ms/step - loss: 0.1179 - accuracy: 0.9944
Epoch 168/500
12/12 [==============================] - 0s 6ms/step - loss: 0.1128 - accuracy: 0.9916
Epoc

12/12 [==============================] - 0s 8ms/step - loss: 0.0066 - accuracy: 1.0000
Epoch 322/500
12/12 [==============================] - 0s 8ms/step - loss: 0.0066 - accuracy: 1.0000
Epoch 323/500
12/12 [==============================] - 0s 7ms/step - loss: 0.0065 - accuracy: 1.0000
Epoch 324/500
12/12 [==============================] - 0s 7ms/step - loss: 0.0064 - accuracy: 1.0000
Epoch 325/500
12/12 [==============================] - 0s 6ms/step - loss: 0.0062 - accuracy: 1.0000
Epoch 326/500
12/12 [==============================] - 0s 7ms/step - loss: 0.0062 - accuracy: 1.0000
Epoch 327/500
12/12 [==============================] - 0s 8ms/step - loss: 0.0061 - accuracy: 1.0000
Epoch 328/500
12/12 [==============================] - 0s 5ms/step - loss: 0.0060 - accuracy: 1.0000
Epoch 329/500
12/12 [==============================] - 0s 7ms/step - loss: 0.0060 - accuracy: 1.0000
Epoch 330/500
12/12 [==============================] - 0s 7ms/step - loss: 0.0060 - accuracy: 1.0000
Epoc

12/12 [==============================] - 0s 7ms/step - loss: 9.8204e-04 - accuracy: 1.0000
Epoch 484/500
12/12 [==============================] - 0s 8ms/step - loss: 9.7140e-04 - accuracy: 1.0000
Epoch 485/500
12/12 [==============================] - 0s 9ms/step - loss: 9.6608e-04 - accuracy: 1.0000
Epoch 486/500
12/12 [==============================] - 0s 9ms/step - loss: 9.5297e-04 - accuracy: 1.0000
Epoch 487/500
12/12 [==============================] - 0s 7ms/step - loss: 9.6305e-04 - accuracy: 1.0000
Epoch 488/500
12/12 [==============================] - 0s 8ms/step - loss: 9.7142e-04 - accuracy: 1.0000
Epoch 489/500
12/12 [==============================] - 0s 8ms/step - loss: 9.6630e-04 - accuracy: 1.0000
Epoch 490/500
12/12 [==============================] - 0s 8ms/step - loss: 9.3996e-04 - accuracy: 1.0000
Epoch 491/500
12/12 [==============================] - 0s 8ms/step - loss: 9.0807e-04 - accuracy: 1.0000
Epoch 492/500
12/12 [==============================] - 0s 7ms/step - 

# Model Preservation

In [10]:
model.save("chat_model")  # Saving the trained model as "chat_model" for future use.

import pickle  # Importing pickle for serialization.

# Saving the fitted tokenizer using pickle serialization:
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Saving the fitted label encoder using pickle serialization:
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(le, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)


INFO:tensorflow:Assets written to: chat_model\assets


INFO:tensorflow:Assets written to: chat_model\assets


# Main

In [11]:
# Import necessary libraries and modules
import requests
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from fuzzywuzzy import fuzz

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

# Load intents from JSON files
with open("intents.json") as file:
    data = json.load(file)
with open("Intent.json",encoding='utf-8') as file:
    data2 = json.load(file)
with open("country.json") as file:
    data3 = json.load(file)
with open("intents_arabic.json",encoding='utf-8') as file:
    data4 = json.load(file)
with open ("health.json") as file:
    data5=json.load(file)
with open ("Booking.json") as file:
    data6=json.load(file)

# Function to start the chatbot
def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20

    while True:
        # Take user input
        inp = input(Fore.LIGHTBLUE_EX + "You: " + Style.RESET_ALL)
        if inp.lower() == "quit":
            print("See you soon my friend")
            break
        
        print(Fore.LIGHTBLUE_EX + "You:" + Style.RESET_ALL, inp)  # Display user input

        # Initialize variables for best response based on input similarity
        best_similarity = -1
        best_response = None

        # Check similarity of input with patterns in the first set of data
        for intent in data['intents']:
            for idx, pattern in enumerate(intent['patterns']):
                if idx < len(intent['responses']):
                    similarity = fuzz.partial_ratio(inp.lower(), pattern.lower())
                    if similarity > best_similarity:
                        best_similarity = similarity
                        best_response = intent['responses'][idx]
        
        # Check similarity of input with patterns in the second set of data
        for intent in data2['intents']:  
            for idx, pattern in enumerate(intent['patterns']):
                if idx < len(intent['responses']):
                    similarity = fuzz.partial_ratio(inp.lower(), pattern.lower())
                    if similarity > best_similarity:
                        best_similarity = similarity
                        best_response = intent['responses'][idx]
        
        # Check similarity of input with patterns in the third set of data
        for intent in data3['intents']:  
            for idx, pattern in enumerate(intent['patterns']):
                if idx < len(intent['responses']):
                    similarity = fuzz.partial_ratio(inp.lower(), pattern.lower())
                    if similarity > best_similarity:
                        best_similarity = similarity
                        best_response = intent['responses'][idx]
        
        # Check similarity of input with patterns in the fourth set of data
        for intent in data4['intents']:
            for idx, pattern in enumerate(intent['patterns']):
                if idx < len(intent['responses']):
                    similarity = fuzz.partial_ratio(inp.lower(), pattern.lower())
                    if similarity > best_similarity:
                        best_similarity = similarity
                        best_response = intent['responses'][idx]
        
        # Check similarity of input with patterns in the fifth set of data
        for intent in data5['intents']:
            for idx, pattern in enumerate(intent['patterns']):
                if idx < len(intent['responses']):
                    similarity = fuzz.partial_ratio(inp.lower(), pattern.lower())
                    if similarity > best_similarity:
                        best_similarity = similarity
                        best_response = intent['responses'][idx]
        
        # Check similarity of input with patterns in the sixth set of data
        for intent in data6['intents']:
            for idx, pattern in enumerate(intent['patterns']):
                if idx < len(intent['responses']):
                    similarity = fuzz.partial_ratio(inp.lower(), pattern.lower())
                    if similarity > best_similarity:
                        best_similarity = similarity
                        best_response = intent['responses'][idx]
        
        # Provide response based on similarity threshold
        if best_similarity > 70 and best_response:  # Set a threshold for similarity
            print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL, best_response)
        else:
            print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL, "Write correctly. You are the one who is wrong. I am definitely correct")




C:\conda11\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


1. Imports: Import necessary libraries/modules for handling requests, JSON data, array operations, machine learning components (TensorFlow Keras for the neural network), label encoding, fuzzy string matching (fuzzywuzzy), and terminal colors using colorama.

2. Loading Intent Data: Loads intent data from multiple JSON files into different variables (data, data2, data3, data4, data5, data6).

3. Chatbot Function (chat()):

    Model Loading: Loads a pre-trained neural network model (chat_model) using Keras' load_model function.
    Tokenizer and Label Encoder Loading: Loads the tokenizer and label encoder objects from their respective pickle files.
    User Interaction Loop: Begins a while loop for interaction with the user until the user inputs 'quit'.
    User Input Processing: Takes user input and processes it.
    Intent Matching with Fuzzy String Matching:
    Checks similarity between user input and patterns within each set of intents (from loaded JSON files) using fuzzy string matching.
    Determines the best response based on the highest similarity score for each set of intents.
    Response Generation:
    Provides responses based on the best similarity score found. If the similarity score is above a certain threshold (70 in this case) and a suitable response is found, the chatbot displays the response.
    If the similarity score is below the threshold or no suitable response is found, it displays a default response indicating an error in the user input.

# Result

In [12]:
# Display a message to inform the user how to close the chat
print(Fore.YELLOW + "if you want to close, type 'Quit'!" + Style.RESET_ALL)
chat()  # Start the chatbot function

if you want to close, type 'Quit'!



You: Hi 
You: Hi 
ChatBot: Hi :)
You: ما هو الفرق بين الجهد والتيار الكهربائي؟
You: ما هو الفرق بين الجهد والتيار الكهربائي؟
ChatBot: الجهد يقاس بالفولت وهو فارق الكهرباء بين نقطتين. التيار يقاس بالأمبير وهو تدفق الكهرباء في الدائرة.
You: ما هو أفضل نوع من الأنابيب للسباكة؟
You: ما هو أفضل نوع من الأنابيب للسباكة؟
ChatBot: البلاستيك والنحاس من أنواع الأنابيب المستخدمة بشكل شائع. اختيار النوع يعتمد على الاحتياجات والبيئة.
You: ما هي أسباب تسرب المياه في الحمام؟
You: ما هي أسباب تسرب المياه في الحمام؟
ChatBot: تسرب المياه في الحمام قد يكون نتيجة لخراب الأختام أو التصاق الجير. يجب إصلاحه فورًا.
You: What is the capital of Antigua and Barbuda?
You: What is the capital of Antigua and Barbuda?
ChatBot: The capital of Antigua and Barbuda is St. John's
You: quit 
You: quit 
ChatBot: I am sorry to disturb you
You: Quit
See you soon my friend
